In [1]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, BayesianRidge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.svm import SVR
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
from tqdm import tqdm
import sys
from pvlib import solarposition
import sys
import os
sys.path.append(os.path.abspath('../common/'))
import common_functions_by_date as cfbd
import datetime
import predictions_algorithmes as pa

In [2]:
weather_path = 'Input/weather_train_set2.csv'
demand_path = 'Input/demand_train_set2.csv'
solar_path = 'Input/pv_train_set2.csv'
dp = cfbd.DataPreprocesser(weather_path, demand_path, solar_path)
dp.load_df()
dp.remove_nan()
dp.interpolate_df()
dp.get_zenith_angle()
dp.get_poa_and_ghi_irradiance()
dp.df.head()

,demand_MW,irradiance_Wm-2,pv_power_mw,panel_temp_C,temp_location3,temp_location6,temp_location2,temp_location4,temp_location5,temp_location1,...,solar_location5,solar_location1,week,dow,date,hour,sp,zenith_angle,GHI,POA
datetime,,,,,,,,,,,,,,,,,,,,,
2017-11-03 00:00:00,2.19,0.0,0.0,7.05,7.46,13.20,9.640,6.680,13.090,8.560,...,0.0,0.0,44,4,2017-11-03,0,1.0,122.821452,0.0,0.0
2017-11-03 00:30:00,2.14,0.0,0.0,7.38,7.30,13.26,9.675,6.475,13.150,8.625,...,0.0,0.0,44,4,2017-11-03,0,2.0,115.780705,0.0,0.0
2017-11-03 01:00:00,2.01,0.0,0.0,7.70,7.14,13.32,9.710,6.270,13.210,8.690,...,0.0,0.0,44,4,2017-11-03,1,3.0,108.668070,0.0,0.0
2017-11-03 01:30:00,1.87,0.0,0.0,7.48,7.00,13.34,9.720,6.090,13.255,8.715,...,0.0,0.0,44,4,2017-11-03,1,4.0,101.505357,0.0,0.0
2017-11-03 02:00:00,1.86,0.0,0.0,7.20,6.86,13.36,9.730,5.910,13.300,8.740,...,0.0,0.0,44,4,2017-11-03,2,5.0,94.307845,0.0,0.0


In [3]:
dp.df.tail()

,demand_MW,irradiance_Wm-2,pv_power_mw,panel_temp_C,temp_location3,temp_location6,temp_location2,temp_location4,temp_location5,temp_location1,...,solar_location5,solar_location1,week,dow,date,hour,sp,zenith_angle,GHI,POA
datetime,,,,,,,,,,,,,,,,,,,,,
2019-03-09 21:30:00,3.20,0.0,0.0,6.83,6.740,9.385,7.865,5.465,9.335,7.80,...,0.0,0.0,10,5,2019-03-09,21,44.0,166.804602,0.0,0.0
2019-03-09 22:00:00,2.97,0.0,0.0,6.99,6.790,9.340,7.930,5.410,9.390,7.79,...,0.0,0.0,10,5,2019-03-09,22,45.0,160.431252,0.0,0.0
2019-03-09 22:30:00,2.69,0.0,0.0,7.14,6.835,9.605,7.955,5.385,9.910,7.89,...,0.0,0.0,10,5,2019-03-09,22,46.0,153.465436,0.0,0.0
2019-03-09 23:00:00,2.51,0.0,0.0,8.20,6.880,9.870,7.980,5.360,10.430,7.99,...,0.0,0.0,10,5,2019-03-09,23,47.0,146.272263,0.0,0.0
2019-03-09 23:30:00,2.33,0.0,0.0,9.54,6.880,9.870,7.980,5.360,10.430,7.99,...,0.0,0.0,10,5,2019-03-09,23,48.0,138.971141,0.0,0.0


In [6]:
first_day_pred=datetime.datetime(2019,3,10).date()

In [7]:
mp = cfbd.MLPredictor(dp,first_day_pred)
mp.get_demand_previous_week()
mp.get_weather_prediction(weather_path)
mp.pred_demand_with_forecast_method_and_average_with_previous_weeks(dp.df, 'Output/rectify_forecast', first_day_pred, 'demand_MW', compute_forecast=True)
mp.predict_pv_power_smooth_and_square_irr()

week prediction with start day :  2019-03-10
  0%|          | 0/336 [00:00<?, ?it/s]2019-03-10 00:00:00
KNeighborsRegressor(n_neighbors=59)
100%|██████████| 336/336 [01:08<00:00,  4.90it/s]
week prediction with start day :  2019-03-03
  0%|          | 0/336 [00:00<?, ?it/s]2019-03-03 00:00:00
KNeighborsRegressor(n_neighbors=39)
100%|██████████| 336/336 [01:13<00:00,  4.55it/s]
week prediction with start day :  2019-02-24
  0%|          | 0/336 [00:00<?, ?it/s]2019-02-24 00:00:00
KNeighborsRegressor(n_neighbors=45)
100%|██████████| 336/336 [01:10<00:00,  4.74it/s]
week prediction with start day :  2019-02-17
  0%|          | 0/336 [00:00<?, ?it/s]2019-02-17 00:00:00
KNeighborsRegressor(n_neighbors=58)
100%|██████████| 336/336 [01:12<00:00,  4.62it/s]
week prediction with start day :  2019-02-10
  0%|          | 0/336 [00:00<?, ?it/s]2019-02-10 00:00:00
KNeighborsRegressor(n_neighbors=59)
100%|██████████| 336/336 [01:13<00:00,  4.58it/s]
smooth_phaze : -3
weather dephasage : 1
smooth_pha

In [8]:
bpd = cfbd.BatteryPowerDispatcher
B = bpd.get_all_dispatch_in_a_week(bpd,mp.predicted_df, first_day_pred)
B_final = bpd.format_dispatching_for_competition(B, mp.predicted_df.index)

In [9]:
B_final.to_csv('Output/BGBattery_last_model_set2.csv')

## Naive prediction

In [10]:
mp.get_demand_previous_week()
mp.get_solar_power_previous_week()

,week,dow,sp,hour,zenith_angle,GHI,POA,demand_MW,temp_location3,temp_location6,...,temp_location4,temp_location5,temp_location1,solar_location3,solar_location6,solar_location2,solar_location4,solar_location5,solar_location1,pv_power_mw
datetime,,,,,,,,,,,,,,,,,,,,,
2019-03-10 00:00:00,10.0,6.0,1.0,0.0,131.610918,0.0,0.0,2.43,7.010,11.220,...,5.830,11.590,9.690,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-03-10 00:30:00,10.0,6.0,2.0,0.0,124.214993,0.0,0.0,2.40,7.740,11.425,...,5.745,11.800,10.450,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-03-10 01:00:00,10.0,6.0,3.0,1.0,116.795915,0.0,0.0,2.28,8.470,11.630,...,5.660,12.010,11.210,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-03-10 01:30:00,10.0,6.0,4.0,1.0,109.360997,0.0,0.0,2.11,9.570,11.690,...,5.785,12.075,11.225,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-03-10 02:00:00,10.0,6.0,5.0,2.0,101.914788,0.0,0.0,2.03,10.670,11.750,...,5.910,12.140,11.240,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-16 21:30:00,11.0,5.0,44.0,21.0,167.940467,0.0,0.0,3.20,6.185,8.755,...,6.760,8.595,6.870,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-03-16 22:00:00,11.0,5.0,45.0,22.0,160.987107,0.0,0.0,2.97,6.030,8.710,...,6.190,8.550,6.730,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-03-16 22:30:00,11.0,5.0,46.0,22.0,153.733325,0.0,0.0,2.69,5.940,8.595,...,5.970,8.440,6.575,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
bpd = cfbd.BatteryPowerDispatcher
B = bpd.get_all_dispatch_in_a_week(bpd,mp.predicted_df, first_day_pred)
B_final = bpd.format_dispatching_for_competition(B, mp.predicted_df.index)

In [12]:
B_final.to_csv('Output/BGBattery_naive_set2.csv')

## Dispatch with real values

In [13]:
weather_path_task3 = '../task3/Input/weather_train_set3.csv'
demand_path_task3 = '../task3/Input/demand_train_set3.csv'
solar_path_task3 = '../task3/Input/pv_train_set3.csv'
dp_task3 = cfbd.DataPreprocesser(weather_path_task3, demand_path_task3, solar_path_task3)
dp_task3.load_df()
dp_task3.remove_nan()
dp_task3.interpolate_df()
# dp.get_zenith_angle()
dp_task3.get_poa_and_ghi_irradiance()
dp_task3.df.head()

,demand_MW,irradiance_Wm-2,pv_power_mw,panel_temp_C,temp_location3,temp_location6,temp_location2,temp_location4,temp_location5,temp_location1,...,solar_location4,solar_location5,solar_location1,week,dow,date,hour,sp,GHI,POA
datetime,,,,,,,,,,,,,,,,,,,,,
2017-11-03 00:00:00,2.19,0.0,0.0,7.05,7.46,13.20,9.640,6.680,13.090,8.560,...,0.0,0.0,0.0,44,4,2017-11-03,0,1.0,0.0,0.0
2017-11-03 00:30:00,2.14,0.0,0.0,7.38,7.30,13.26,9.675,6.475,13.150,8.625,...,0.0,0.0,0.0,44,4,2017-11-03,0,2.0,0.0,0.0
2017-11-03 01:00:00,2.01,0.0,0.0,7.70,7.14,13.32,9.710,6.270,13.210,8.690,...,0.0,0.0,0.0,44,4,2017-11-03,1,3.0,0.0,0.0
2017-11-03 01:30:00,1.87,0.0,0.0,7.48,7.00,13.34,9.720,6.090,13.255,8.715,...,0.0,0.0,0.0,44,4,2017-11-03,1,4.0,0.0,0.0
2017-11-03 02:00:00,1.86,0.0,0.0,7.20,6.86,13.36,9.730,5.910,13.300,8.740,...,0.0,0.0,0.0,44,4,2017-11-03,2,5.0,0.0,0.0


In [14]:
real_value_df = dp_task3.df[(dp_task3.df.index.date >= first_day_pred) & (dp_task3.df.index.date < first_day_pred + datetime.timedelta(days=7))]

In [16]:
bpd = cfbd.BatteryPowerDispatcher
B_real = bpd.get_all_dispatch_in_a_week(bpd,real_value_df, first_day_pred)
B_final_real = bpd.format_dispatching_for_competition(B_real, real_value_df.index)

In [17]:
B_final_real.to_csv('Output/BGBattery_real_set2.csv')